In [544]:
### 모듈 로딩
from PIL import Image
import torchvision.transforms as transforms 
from torchvision.datasets import ImageFolder 
### ===> Module Import
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torchvision import transforms, datasets
from torch.utils.data import Dataset,DataLoader,WeightedRandomSampler
import torch.optim as optim
from torchinfo import summary

In [545]:
import os
import random
from shutil import copy2

파일 전처리 함수 만들기

In [546]:
from torchvision.datasets import ImageFolder 

img_root = "./pizza_not_pizza/" # 해당 경로 내에 있는 파일명이 곧 label이 되는 것
train_root="./pizza_not_pizza/train"
test_root="./pizza_not_pizza/test"
validation_root="./pizza_not_pizza/validation"



mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

preprocessing = transforms.Compose([
    transforms.Resize((227,227), interpolation=transforms.InterpolationMode.BILINEAR), # 1. resize
    transforms.ToTensor(),  # 3. 값의 크기를 0~1로
    transforms.Normalize(mean=mean, std=std) # 4. normalized
])

imgDS = ImageFolder(root=img_root, transform=preprocessing)
validDS = ImageFolder(root=validation_root, transform=preprocessing)
testDS = ImageFolder(root=test_root, transform=preprocessing)
trainDS = ImageFolder(root=train_root, transform=preprocessing)


이미지 ds 클래스 확인, 클래스를 idx로 

In [569]:
print(len(imgDS),len(trainDS),len(testDS),len(validDS))

1966 1600 166 200


In [547]:
validDS.classes,validDS.class_to_idx

(['not_pizza', 'pizza'], {'not_pizza': 0, 'pizza': 1})

In [548]:
# weights = make_weights(imgDS.targets, len(imgDS.classes))
# weights = torch.DoubleTensor(weights)

이미지 샘플러

In [549]:
# img_sampler = WeightedRandomSampler(weights, len(weights))

In [550]:
plt.imshow(imgDS[400][1].permute(1,2,0))

AttributeError: 'int' object has no attribute 'permute'

In [551]:
batch_size = 16
trainDL = DataLoader(
    trainDS, batch_size=batch_size, drop_last=True,shuffle=True
)
validDL = DataLoader(
    validDS, batch_size=batch_size, drop_last=True,shuffle=True
)

testDL = DataLoader(
    testDS, batch_size=batch_size, drop_last=True,shuffle=True
)

[ 이미지랑 라벨 쪽 형태 보기]

In [552]:
### ===> 딥러닝 모델을 설계할 때 활용하는 장비 확인
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 2.2.0  Device: cpu


In [553]:
len(imgDS), len(trainDS)/40, type(trainDL) 

(1966, 40.0, torch.utils.data.dataloader.DataLoader)

In [554]:
### ===>  데이터 확인하기 (1)
for (X_train, y_train) in trainDL:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    

X_train: torch.Size([16, 3, 227, 227]) type: torch.FloatTensor
y_train: torch.Size([16]) type: torch.LongTensor
X_train: torch.Size([16, 3, 227, 227]) type: torch.FloatTensor
y_train: torch.Size([16]) type: torch.LongTensor
X_train: torch.Size([16, 3, 227, 227]) type: torch.FloatTensor
y_train: torch.Size([16]) type: torch.LongTensor
X_train: torch.Size([16, 3, 227, 227]) type: torch.FloatTensor
y_train: torch.Size([16]) type: torch.LongTensor
X_train: torch.Size([16, 3, 227, 227]) type: torch.FloatTensor
y_train: torch.Size([16]) type: torch.LongTensor
X_train: torch.Size([16, 3, 227, 227]) type: torch.FloatTensor
y_train: torch.Size([16]) type: torch.LongTensor
X_train: torch.Size([16, 3, 227, 227]) type: torch.FloatTensor
y_train: torch.Size([16]) type: torch.LongTensor
X_train: torch.Size([16, 3, 227, 227]) type: torch.FloatTensor
y_train: torch.Size([16]) type: torch.LongTensor
X_train: torch.Size([16, 3, 227, 227]) type: torch.FloatTensor
y_train: torch.Size([16]) type: torch.Lon

In [555]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__() 
        self.conv1 = nn.Conv2d(in_channels =3, out_channels = 8, kernel_size = 3, padding = 1) 
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, padding = 1)
        
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.fc1 = nn.Linear(
            # 28*28*16,
            50176,
            128
        )
        self.fc2 =nn.Linear(128,64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1) 
        
    def forward(self, x):
        x = self.conv1(x) 
        x = F.relu(x)
        x = self.pool(x) 
        x = self.conv2(x) 
        x = F.relu(x)
        x = self.pool(x) 
        
        # x = x.view(-1, 28*28*16)
        x = nn.Flatten()(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x=F.relu(x)
        x=self.fc4(x)
        
        return F.sigmoid(x)

In [556]:
### ===> Optimizer, Objective Function 설정
from torch.optim.lr_scheduler import ReduceLROnPlateau
model = CNN().to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001) #epoch마다 학습률을 0.001씩 줄임
scheduler=ReduceLROnPlateau(optimizer,'min',patience=4)
# criterion = nn.CrossEntropyLoss() # 손실함수
# criterion = nn.BCELoss()
criterion=nn.BCELoss()
print(model)

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)


In [557]:
# CNN 모델 학습 진행 함수
def train(epoch, model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        # print(image.shape, label.shape)
        image = image.to(DEVICE)
        # print(image.shape, label.shape)
        
        
        label = label.to(DEVICE).unsqueeze(dim=1).float()
        output = model(image)
        # print(output.shape, label.shape)

        
        # print(output.shape)
        # for p in model.parameters():
        #     print(p)
        # output_prob = F.sigmoid(output)
        output_prob = output
        # print(output_prob,label)
        # print(output_prob.size(),label.size())
        
        # print(output_prob.shape, label.shape)
        loss = criterion(output_prob, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx}]\tTrain Loss: {loss.item():.6f}")
    


In [558]:

### 검증 validation 진행 함수 
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            # print(image, label)
            image = image.to(DEVICE)
            label = label.to(DEVICE).unsqueeze(dim=1).float()
            
            
            output = model(image)
            output_prob = F.sigmoid(output)
            #test_loss
            test_loss += criterion(output_prob, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
            
        
    
    
    test_loss /= len(testDL.dataset)
    test_accuracy = 100. * correct / len(testDL.dataset)
    return test_loss, test_accuracy

In [559]:
# CNN 학습
EPOCHS =20

for epoch in range(1, EPOCHS + 1):
    
    train(epoch, model,trainDL, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, testDL)
    
    print(f"\n[EPOCH: {epoch}], \tTest Loss: {test_loss:.4f}, \tTest Accuracy: {test_accuracy:.2f} % \n")
    
    

Train Epoch: 1 [0]	Train Loss: 0.760439

[EPOCH: 1], 	Test Loss: 0.0439, 	Test Accuracy: 48.80 % 

Train Epoch: 2 [0]	Train Loss: 0.697082

[EPOCH: 2], 	Test Loss: 0.0439, 	Test Accuracy: 48.80 % 
Train Epoch: 3 [0]	Train Loss: 0.627490

[EPOCH: 3], 	Test Loss: 0.0434, 	Test Accuracy: 48.19 % 

Train Epoch: 4 [0]	Train Loss: 0.555552

[EPOCH: 4], 	Test Loss: 0.0433, 	Test Accuracy: 48.19 % 
Train Epoch: 5 [0]	Train Loss: 0.623226

[EPOCH: 5], 	Test Loss: 0.0428, 	Test Accuracy: 47.59 % 
Train Epoch: 6 [0]	Train Loss: 0.611180

[EPOCH: 6], 	Test Loss: 0.0434, 	Test Accuracy: 48.80 % 
Train Epoch: 7 [0]	Train Loss: 0.572760

[EPOCH: 7], 	Test Loss: 0.0428, 	Test Accuracy: 47.59 % 
Train Epoch: 8 [0]	Train Loss: 0.532921

[EPOCH: 8], 	Test Loss: 0.0437, 	Test Accuracy: 49.40 % 
Train Epoch: 9 [0]	Train Loss: 0.475795

[EPOCH: 9], 	Test Loss: 0.0433, 	Test Accuracy: 48.80 % 
Train Epoch: 10 [0]	Train Loss: 0.589183

[EPOCH: 10], 	Test Loss: 0.0435, 	Test Accuracy: 49.40 % 
Train Epoch: 11 

### ALEXNET 사용

In [570]:
from torchvision.models import resnet18,ResNet18_Weights,AlexNet


#데이터 모델 학습 준비 
model = AlexNet(num_classes=1)# 클래스 수에 맞춰 출력 레이어 수정(클래스 수 1개로)
# model.classifier.append(nn.Linear(2,1))
model.__dir__()
for p in model.named_children():
    print(p)
model = model.to(DEVICE)

# 손실 함수와 옵티마이저 정의
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# 데이터 재설정 

ALEXPREPRO = transforms.Compose([
    transforms.Resize(256),  # 이미지를 리사이징
    transforms.CenterCrop(224),  # 이미지 중심부 자르기
    transforms.ToTensor()
])

# 데이터셋과 데이터로더 생성
ALEXvalidDS = ImageFolder(root=validation_root, transform=ALEXPREPRO)
ALEXtestDS = ImageFolder(root=test_root, transform=ALEXPREPRO)
ALEXtrainDS = ImageFolder(root=train_root, transform=ALEXPREPRO)

ALEXtrainDL = DataLoader(ALEXtrainDS, batch_size=32, shuffle=True)
ALEXvalidDL = DataLoader(ALEXvalidDS, batch_size=32, shuffle=False)
ALEXtestDL = DataLoader(ALEXtestDS, batch_size=32, shuffle=False)



# CNN 학습
EPOCHS =20

for epoch in range(1, EPOCHS + 1):

    train(epoch, model, ALEXtrainDL, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, ALEXtestDL)

    print(f"\n[EPOCH: {epoch}], \tTest Loss: {test_loss:.4f}, \tTest Accuracy: {test_accuracy:.2f} % \n")



('features', Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
))
('avgpool', AdaptiveAvgPool2d(output_size=(6, 6)))
('classifier', Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=

In [561]:
# summary(model)

In [562]:
# for p in model.named_parameters():
#     print(p)

In [563]:
# model.__dir__()

In [564]:
# for p in model.named_children():
#     print(p)

In [565]:
# model.classifier[-1].out_features = 1
# model.classifier[-1]

In [566]:
# 모델, optimizer 상태도 저장
torch.save(model.state_dict(), 'model')